In [1]:
import numpy as np
import xgboost as xgb
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error
import optuna
import pickle
from datetime import timedelta
import time
import os 

In [2]:
feat = pd.read_csv('./data/test.csv')
FEATURE_COLS = feat.columns[1:].tolist()

In [3]:
study_name = '426_convnextbase_003_998_1'

In [4]:
pickle_file_path = f'./data/test_{study_name}.pickle'

with open(pickle_file_path, 'rb') as f:
    test_df = pickle.load(f)

pickle_file_path = f'./data/train_{study_name}.pickle'

with open(pickle_file_path, 'rb') as f:
    train_df = pickle.load(f)

In [5]:
def get_combined_data(df):
    # Oletetaan, että FEATURES_COLS on jo määritelty olemassa oleville piirteille
    data = [df[col].values for col in FEATURE_COLS]
    # Lisää mallin piirteet
    data.append(np.vstack(df['combined_features'].values))
    return np.column_stack(data)

def objective(trial, df, target, fold_train, fold_validation):
    param = {        
        'objective': 'reg:squarederror',        
        'device' : 'cuda',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log = True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log = True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.9),
        'subsample': trial.suggest_float('subsample', 0.1, 0.9),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0,),
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000, log = True),
        'max_depth': trial.suggest_int('max_depth', 2, 20),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 100),
        'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 5, 50)}
    

    train_data = df[df['fold'] == fold_train]
    valid_data = df[df['fold'] == fold_validation]

    X_train = get_combined_data(train_data)
    X_valid = get_combined_data(valid_data)

    y_train = train_data[target]
    y_valid = valid_data[target]

    model = xgb.XGBRegressor(**param)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=0)
    preds = model.predict(X_valid)
    mse = mean_squared_error(y_valid, preds)

    return mse

def optimize_model(df, target, fold_train, fold_validation):

    if os.path.exists(f'./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_genesampler.pickle'):            
        with open(f'./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_genesampler.pickle', 'rb') as f:
            print(f'Loading gene sampler from file {f}')
            genemachine = pickle.load(f)

    else:            
        print('Creating new gene sampler')
        genemachine = optuna.samplers.NSGAIISampler(crossover = optuna.samplers.nsgaii.VSBXCrossover())

    if os.path.exists(f'./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_qmc_sampler.pickle'):
        with open(f'./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_qmc_sampler.pickle', 'rb') as f:
            print(f'Loading QMC sampler from file {f}')
            qmc_sampler = pickle.load(f)
    else:
        print(f'Creating new QMC sampler')
        qmc_sampler = optuna.samplers.QMCSampler(warn_independent_sampling = False)

    if os.path.exists(f'./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_tpe_sampler.pickle'):
        with open(f'./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_tpe_sampler.pickle', 'rb') as f:
            print(f'Loading TPE sampler from file {f}')
            tpe_sampler = pickle.load(f)
    else:
        print(f'Creating new TPE sampler')
        tpe_sampler = optuna.samplers.TPESampler(n_startup_trials=0, multivariate=True, warn_independent_sampling = False)


    start_time = time.time()
    study = optuna.create_study(direction='minimize',
                            study_name=study_name,
                            storage=f'sqlite:///427_xgboost_{target}.db',
                            load_if_exists=True                            
                            )
    
    print(f'Starting optimization for {target} with qmc sampler')
    random_time = time.time()
    study.sampler = qmc_sampler
    study.optimize(lambda trial: objective(trial, df, target, fold_train, fold_validation), n_trials=42)
    print(f'QCM optimization finished in {timedelta(seconds=time.time() - random_time)}')

    print(f'Saving QMC sampler to file ./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_qmc_sampler.pickle')
    with open(f'./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_qmc_sampler.pickle', 'wb') as f:
        pickle.dump(qmc_sampler, f)

    print(f'Starting optimization for {target} with gene sampler')
    gene_time = time.time()
    study.sampler = genemachine
    study.optimize(lambda trial: objective(trial, df, target, fold_train, fold_validation), n_trials=42)
    print(f'Gene optimization finished in {timedelta(seconds=time.time() - gene_time)}')

    print(f'Saving gene sampler to file ./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_genesampler.pickle')
    with open(f'./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_genesampler.pickle', 'wb') as f:
        pickle.dump(genemachine, f)

    print(f'Starting optimization for {target} with TPE sampler')
    tpe_time = time.time()
    study.sampler = tpe_sampler
    study.optimize(lambda trial: objective(trial, df, target, fold_train, fold_validation), n_trials=42)
    print(f'TPE optimization finished in {timedelta(seconds=time.time() - tpe_time)}')

    print(f'Saving TPE sampler to file ./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_tpe_sampler.pickle')
    with open(f'./NN_search/{study_name}_{fold_train}_{fold_validation}_{target}_tpe_sampler.pickle', 'wb') as f:
        pickle.dump(tpe_sampler, f)

    print(f'Optimization finished in {timedelta(seconds=time.time() - start_time)}')

    best_params = study.best_trial.params    

    mse_scores = []

    for fold in df['fold'].unique():
        train_data = df[df['fold'] != fold]
        valid_data = df[df['fold'] == fold]

        X_train = get_combined_data(train_data)
        X_valid = get_combined_data(valid_data)

        y_train = train_data[target]
        y_valid = valid_data[target]

    
        model = xgb.XGBRegressor(**best_params)
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=0)
        preds = model.predict(X_valid)
        mse = mean_squared_error(y_valid, preds)
        mse_scores.append(mse)

    average_mse = np.mean(mse_scores)

    print(f'Cross-validated MSE: {average_mse} for {target}')

    if 'early_stopping_rounds' in best_params:
        del best_params['early_stopping_rounds']
    print(f"Best parameters for {target}: ", best_params)

    print(f'Fitting model for {target} with best parameters')
    
    # Käytä parhaita parametreja koko datan kouluttamiseen
    X = get_combined_data(df)
    y = df[target]
    model = xgb.XGBRegressor(**best_params)
    model.fit(X, y)
    
    return model



In [6]:
train_df.head()

,id,WORLDCLIM_BIO1_annual_mean_temperature,WORLDCLIM_BIO12_annual_precipitation,WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month,WORLDCLIM_BIO15_precipitation_seasonality,WORLDCLIM_BIO4_temperature_seasonality,WORLDCLIM_BIO7_temperature_annual_range,SOIL_bdod_0.5cm_mean_0.01_deg,SOIL_bdod_100.200cm_mean_0.01_deg,SOIL_bdod_15.30cm_mean_0.01_deg,...,bin_0,bin_1,bin_2,bin_3,bin_4,bin_5,final_bin,fold,features_avg,model_features_426_convnextbase_003_998_1_finetuned
0,192027691,12.235703,374.466675,62.524445,72.256844,773.592041,33.277779,125,149,136,...,2,2,1,4,2,1,221421,2.0,"[-0.14304829, -0.28004962, 0.886131, -0.183084...","[-0.3265194594860077, -0.5460191965103149, -0...."
1,195542235,17.270555,90.239998,10.351111,38.220940,859.193298,40.009777,124,144,138,...,3,3,2,2,2,3,332223,4.0,"[0.16563013, -1.4509088, 0.46862665, 0.2771467...","[0.009429454803466797, -0.5460116267204285, -0..."
2,196639184,14.254504,902.071411,49.642857,17.873655,387.977753,22.807142,107,133,119,...,5,1,5,5,2,3,515523,2.0,"[-0.034171782, -0.13625506, 0.5971655, -0.6490...","[-0.1011316254734993, -0.5460186004638672, -0...."
3,195728812,18.680834,1473.933350,163.100006,45.009758,381.053986,20.436666,120,131,125,...,3,2,3,2,1,3,323213,0.0,"[-0.881118, -0.16722384, 1.3514438, 0.34779415...","[1.4968960285186768, -0.5460196137428284, -0.5..."
4,195251545,0.673204,530.088867,50.857777,38.230709,1323.526855,45.891998,91,146,120,...,2,3,3,5,4,4,233544,4.0,"[-0.3382826, 0.3086146, 0.58881557, 0.24890125...","[-0.3912062346935272, -0.5460014343261719, -0...."


In [7]:
def prepare_features(row):
    return np.array(row[f'model_features_{study_name}_finetuned'])

train_df['combined_features'] = train_df.apply(prepare_features, axis=1)
test_df['combined_features'] = test_df.apply(prepare_features, axis=1)



In [8]:
target_columns = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']
train_fold = 1
validation_fold = 2

# Mallien kouluttaminen jokaiselle kohdemuuttujalle
models = {}

time_search_start = time.time()
time_taken = 0

while time_taken < 3600 * 8:
    for target in target_columns:    
        print(f'\n\nOptimizing model for {target} using train fold {train_fold} and validation fold {validation_fold}\n\n')
        models[target] = optimize_model(train_df, target, train_fold, validation_fold)
        time_taken = time.time() - time_search_start
        print(f'Time taken: {timedelta(seconds=time_taken)}')   



Optimizing model for X4_mean using train fold 1 and validation fold 2


Creating new gene sampler
Creating new QMC sampler
Creating new TPE sampler


/tmp/ipykernel_3655715/1750598810.py:48: ExperimentalWarning: VSBXCrossover is experimental (supported from v3.0.0). The interface can change in the future.
  genemachine = optuna.samplers.NSGAIISampler(crossover = optuna.samplers.nsgaii.VSBXCrossover())
/tmp/ipykernel_3655715/1750598810.py:56: ExperimentalWarning: QMCSampler is experimental (supported from v3.0.0). The interface can change in the future.
  qmc_sampler = optuna.samplers.QMCSampler(warn_independent_sampling = False)
/home/tobias/miniconda3/envs/tf/lib/python3.9/site-packages/optuna/samplers/_tpe/sampler.py:319: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2024-04-27 23:19:08,587] A new study created in RDB with name: 426_convnextbase_003_998_1


Starting optimization for X4_mean with qmc sampler


/home/tobias/miniconda3/envs/tf/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [23:19:12] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1705650282415/work/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
[I 2024-04-27 23:19:12,092] Trial 0 finished with value: 0.0072839706961062775 and parameters: {'lambda': 6.037116720945384e-07, 'alpha': 0.012774407106202087, 'colsample_bytree': 0.20788578187445517, 'subsample': 0.3601674899569792, 'learning_rate': 0.3784249525690579, 'n_estimators': 372, 'max_depth': 14, 'min_child_weight': 86, 'early_stopping_rounds': 8

In [ ]:
features_array = np.array(train_df['combined_features'].tolist())
X_combined_train = np.hstack([train_df[FEATURE_COLS].values, features_array])

train_pred = np.zeros((train_df.shape[0], len(target_columns)))

for i, target in enumerate(target_columns):    
    train_pred[:, i] = models[target].predict(X_combined_train)

train_r2 = r2_score(df[target_columns], train_pred)
print(f'Training R2: {train_r2}')

In [ ]:
features_array = np.array(test_df['combined_features'].tolist())
X_combined_test = np.hstack([test_df[FEATURE_COLS].values, features_array])

test_preds = np.zeros((len(test_df), len(target_columns)))

for i, target in enumerate(target_columns):
    print(f'Predicting {target} with model {models[target]}')
    test_preds[:, i] = models[target].predict(X_combined_test)
     

In [ ]:
target_columns = ['X4', 'X11', 'X18', 'X50', 'X26', 'X3112']

test_df_copy = test_df.copy()
submission_df = test_df_copy[['id']].copy()
submission_df[target_columns] = test_preds

In [ ]:
submission_df.describe()

In [ ]:
train_df[target_columns].describe()

In [ ]:
submission_df.head()

In [ ]:
submission_df.to_csv('./data/submission.csv', index=False)